In [1]:
pwd

'/content'

In [2]:
import os
import shutil
import zipfile
import random
import csv
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.models import load_model
from sklearn import preprocessing
from tensorflow.keras.preprocessing import image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tqdm import tqdm
import matplotlib.pyplot as plt
from PIL import Image
import cv2
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.preprocessing import image
import numpy as np



In [3]:
cd /content/drive/MyDrive/BL

/content/drive/MyDrive/BL


In [4]:
# Loading the model

# Loading the Model

In [5]:
loaded_model = load_model('xception_covid.h5')

In [6]:
loaded_model.save('xception_covid.h5')


!tar -zcvf ibm_covid_prediction.tgz  xception_covid.h5

xception_covid.h5


In [27]:
!pip install watson-machine-learning-client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [28]:
!pip install watson-machine-learning-client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [29]:

!pip install ibm_watson_machine_learning
     

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [46]:

from ibm_watson_machine_learning import APIClient

wml_credentials = {
    "url":"https://us-south.ml.cloud.ibm.com",
    "apikey":"ToLs9CHbNLXNFUrM8Rv954IYUoEq-mE7DYnxfVqBBgB_"
}
     

In [47]:
client = APIClient(wml_credentials)

In [48]:
client

In [49]:

client.spaces.get_details()

{'resources': [{'entity': {'description': '',
    'name': 'Covid',
    'scope': {'bss_account_id': 'daa88c6792d74da8bf74a7978080ffd0'},
    'stage': {'production': False},
    'status': {'state': 'active'},
    'storage': {'properties': {'bucket_name': '1e6e664e-0f5b-4269-a04a-f403d97a3543',
      'bucket_region': 'us-south',
      'credentials': {'admin': {'access_key_id': '859445b41e794d60ba860e2cba14a47c',
        'api_key': 'GD6bU_rC4KpOTqthvcUrkFSqzGu0HOilX4NDmSWOzNv8',
        'secret_access_key': '62c5f9c11fa8a72d64d217e9910351112a4bcb0ad5cc2d6f',
        'service_id': 'ServiceId-2e552a13-1fc3-4ca9-872c-32eeb2ca8391'},
       'editor': {'access_key_id': '42dc112c06114c26ba1e6a5dd608bb0f',
        'api_key': 'swVUfdf2jC1BInDuHaYeNneRCwUOvVXF7UwhgEPZ1cUl',
        'resource_key_crn': 'crn:v1:bluemix:public:cloud-object-storage:global:a/daa88c6792d74da8bf74a7978080ffd0:cd8572ee-7992-47d6-a8b3-e4644d997762::',
        'secret_access_key': '710e112c5966b36c1cd8073d34fa6ab8fbb202b726c

In [50]:
client.spaces.list()

Note: 'limit' is not provided. Only first 50 records will be displayed if the number of records exceed 50
------------------------------------  -----  ------------------------
ID                                    NAME   CREATED
7fa55e43-82b9-4412-8b74-6abc70fc3aec  covid  2023-05-20T09:16:40.800Z
f2ca1b9e-59a5-4455-90f8-68577f5851f9  Covid  2023-05-20T03:16:25.859Z
------------------------------------  -----  ------------------------


,ID,NAME,CREATED
0,7fa55e43-82b9-4412-8b74-6abc70fc3aec,covid,2023-05-20T09:16:40.800Z
1,f2ca1b9e-59a5-4455-90f8-68577f5851f9,Covid,2023-05-20T03:16:25.859Z


In [51]:
space_uid = "7fa55e43-82b9-4412-8b74-6abc70fc3aec"
space_uid

'7fa55e43-82b9-4412-8b74-6abc70fc3aec'

In [52]:
client.set.default_space(space_uid)

'SUCCESS'

In [63]:
client.software_specifications.list(200)

-------------------------------  ------------------------------------  ----  ------------------  --------------------------
NAME                             ID                                    TYPE  STATE               REPLACEMENT
default_py3.6                    0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base  unsupported         runtime-22.2-py3.10
autoai-ts_rt23.1-py3.10          01ce9391-1a79-5a33-94fb-2e134337f314  base  not_provided
kernel-spark3.2-scala2.12        020d69ce-7ac1-5e68-ac1a-31189867356a  base  not_provided
pytorch-onnx_1.3-py3.7-edt       069ea134-3346-5748-b513-49120e15d288  base  not_provided
tensorflow_rt23.1-py3.10         079a91e0-245f-5269-8926-3c20b28f37dc  base  not_provided
scikit-learn_0.20-py3.6          09c5a1d0-9c1e-4473-a344-eb7b665ff687  base  unsupported         runtime-22.2-py3.10
spark-mllib_3.0-scala_2.12       09f4cff0-90a7-5899-b9ed-1ef348aebdee  base  unsupported
pytorch-onnx_rt22.1-py3.9        0b848dd4-e681-5599-be41-b5f6fccc6471  base  create-

,NAME,ID,TYPE,STATE,REPLACEMENT
0,default_py3.6,0062b8c9-8b7d-44a0-a9b9-46c416adcbd9,base,unsupported,runtime-22.2-py3.10
1,autoai-ts_rt23.1-py3.10,01ce9391-1a79-5a33-94fb-2e134337f314,base,not_provided,
2,kernel-spark3.2-scala2.12,020d69ce-7ac1-5e68-ac1a-31189867356a,base,not_provided,
3,pytorch-onnx_1.3-py3.7-edt,069ea134-3346-5748-b513-49120e15d288,base,not_provided,
4,tensorflow_rt23.1-py3.10,079a91e0-245f-5269-8926-3c20b28f37dc,base,not_provided,
...,...,...,...,...,...
103,runtime-22.2-r4.2,ec0a3d28-08f7-556c-9674-ca7c2dba30bd,base,supported,
104,tensorflow_rt22.2-py3.10,f65bd165-f057-55de-b5cb-f97cf2c0f393,base,supported,
105,do_20.1,f686cdd9-7904-5f9d-a732-01b0d6b10dc5,base,supported,
106,scikit-learn_0.19-py3.6,f963fa9d-4bb7-5652-9c5d-8d9289ef6ad9,base,unsupported,runtime-22.2-py3.10


In [91]:
software_space_uid = client.software_specifications.get_uid_by_name("tensorflow_rt22.2-py3.10")
software_space_uid
 

'f65bd165-f057-55de-b5cb-f97cf2c0f393'

In [83]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [84]:
import tensorflow as tf
tf.__version__

'2.12.0'

In [85]:
ls

ibm_covid_prediction.tgz  Untitled0.ipynb  xception_covid.h5


In [92]:
model = client.repository.store_model(model="ibm_covid_prediction.tgz",
                                      meta_props={
    client.repository.ModelMetaNames.NAME:"covid_prediction",
    client.repository.ModelMetaNames.TYPE:"tensorflow_2.9",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_space_uid})
     

In [93]:
model

{'entity': {'hybrid_pipeline_software_specs': [],
  'software_spec': {'id': 'f65bd165-f057-55de-b5cb-f97cf2c0f393',
   'name': 'tensorflow_rt22.2-py3.10'},
  'type': 'tensorflow_2.9'},
 'metadata': {'created_at': '2023-05-20T10:04:02.424Z',
  'id': '22f3bc04-9c71-42cd-8433-1d786f45766d',
  'modified_at': '2023-05-20T10:04:17.477Z',
  'name': 'covid_prediction',
  'owner': 'IBMid-6650032SQ9',
  'resource_key': 'ed9e6121-2a49-4d6f-8b89-689cbe68d3fd',
  'space_id': '7fa55e43-82b9-4412-8b74-6abc70fc3aec'},
 'system': {'warnings': []}}

In [94]:
model_id = client.repository.get_model_id(model)
model_id

'22f3bc04-9c71-42cd-8433-1d786f45766d'

In [96]:
client.repository.download(model_id,'xception_covid.tgz')

Successfully saved model content to file: 'xception_covid.tgz'


'/content/drive/MyDrive/BL/xception_covid.tgz'